In [4]:
import datetime as dt
import itertools
import numpy as np
import pandas as pd
import os
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
import holidays


from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot
from plotnine import *

In [5]:
# BASE_DIR = Path(__name__).resolve().parent.parent
FILES_DIR = '../files/'
bidding_df = pd.read_csv(FILES_DIR + 'bidding_results.csv')
log_df = pd.read_csv(FILES_DIR + 'log_csv.csv')

In [6]:
bid_data = bidding_df.copy()
bid_data

,index,Impression,Click,Cost,Sum of AD rank,Bid,Price(PC),Price(Mobile),Delivery Fee,Category Name 1,Category Name 2,Category Name 3,Category Name 4,date
0,3575092,3,0,0.0,3.000000,50,165900,165900,0,가구/인테리어,침실가구,장롱/붙박이장,드레스룸,20221031
1,3575637,6,0,0.0,3.333333,50,10900,10900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,20221031
2,4129259,7,0,0.0,8.000000,50,255000,255000,-1,가구/인테리어,거실가구,소파,패브릭소파,20221031
3,4131479,36,3,150.0,4.416667,140,190000,190000,20000,가구/인테리어,아동/주니어가구,책상의자세트,NaN,20221031
4,4131482,224,0,0.0,5.696429,50,129000,129000,0,가구/인테리어,아동/주니어가구,책상,NaN,20221031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337083,2566795,5,0,0.0,13.200000,500,19900,19900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,20230103
1337084,2566880,3,0,0.0,17.000000,500,23900,23900,3000,생활/건강,문구/사무용품,이벤트/파티용품,데코용품,20230103
1337085,2566881,1,0,0.0,8.000000,500,2500,2500,3000,가구/인테리어,인테리어소품,기타장식용품,NaN,20230103
1337086,2566891,2,0,0.0,4.000000,500,21000,21000,3000,생활/건강,정원/원예용품,화분받침,NaN,20230103


In [7]:
# CTR = 클릭률
# CPC = 클릭당 비용
# CPM = 1000회 노출당 비용
# Sum of AD rank = 반올림
bid_data["CTR"] = (bid_data["Click"]/bid_data["Impression"]) * 100
bid_data["CPM"] = (bid_data["Cost"]/1000)
bid_data['Sum of AD rank'] = bid_data['Sum of AD rank'].round(0)

In [8]:
def _trans_date_time(df, date):
    df.index = pd.to_datetime(df[date], format='%Y%m%d')
    df.drop(date, axis=1, inplace=True)
    return df

_trans_date_time(df=bid_data, date='date')

,index,Impression,Click,Cost,Sum of AD rank,Bid,Price(PC),Price(Mobile),Delivery Fee,Category Name 1,Category Name 2,Category Name 3,Category Name 4,CTR,CPM
date,,,,,,,,,,,,,,,
2022-10-31,3575092,3,0,0.0,3.0,50,165900,165900,0,가구/인테리어,침실가구,장롱/붙박이장,드레스룸,0.000000,0.00
2022-10-31,3575637,6,0,0.0,3.0,50,10900,10900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,0.000000,0.00
2022-10-31,4129259,7,0,0.0,8.0,50,255000,255000,-1,가구/인테리어,거실가구,소파,패브릭소파,0.000000,0.00
2022-10-31,4131479,36,3,150.0,4.0,140,190000,190000,20000,가구/인테리어,아동/주니어가구,책상의자세트,NaN,8.333333,0.15
2022-10-31,4131482,224,0,0.0,6.0,50,129000,129000,0,가구/인테리어,아동/주니어가구,책상,NaN,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-03,2566795,5,0,0.0,13.0,500,19900,19900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,0.000000,0.00
2023-01-03,2566880,3,0,0.0,17.0,500,23900,23900,3000,생활/건강,문구/사무용품,이벤트/파티용품,데코용품,0.000000,0.00
2023-01-03,2566881,1,0,0.0,8.0,500,2500,2500,3000,가구/인테리어,인테리어소품,기타장식용품,NaN,0.000000,0.00


In [9]:
bid_data_click = bid_data[bid_data['Click'] != 0]
bid_data_click

,index,Impression,Click,Cost,Sum of AD rank,Bid,Price(PC),Price(Mobile),Delivery Fee,Category Name 1,Category Name 2,Category Name 3,Category Name 4,CTR,CPM
date,,,,,,,,,,,,,,,
2022-10-31,4131479,36,3,150.0,4.0,140,190000,190000,20000,가구/인테리어,아동/주니어가구,책상의자세트,NaN,8.333333,0.15
2022-10-31,4158472,138,2,270.0,4.0,240,129000,129000,0,가구/인테리어,아동/주니어가구,책장,NaN,1.449275,0.27
2022-10-31,4159062,60,1,50.0,10.0,50,699000,699000,0,가구/인테리어,아동/주니어가구,책상,NaN,1.666667,0.05
2022-10-31,4159066,76,3,640.0,5.0,250,149000,149000,0,가구/인테리어,아동/주니어가구,책상,NaN,3.947368,0.64
2022-10-31,4159664,170,2,200.0,3.0,230,359000,359000,-1,가구/인테리어,침실가구,침대,침대프레임,1.176471,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-03,2564155,5,1,190.0,15.0,50,98900,98900,-1,가구/인테리어,침실가구,침대,침대프레임,20.000000,0.19
2023-01-03,2564724,39,1,250.0,3.0,500,6900,6900,3000,화장품/미용,바디케어,핸드케어,NaN,2.564103,0.25
2023-01-03,2566230,85,2,860.0,2.0,500,36800,36800,0,생활/건강,욕실용품,수건/타월,세면타월,2.352941,0.86


In [10]:
bid_data_click.dropna(axis = 0, inplace = True)

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\1008849750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
def missing_val(df):
  print(df.info())
  print('\n\nFeatures with missing values in descending order \n\n')
  print((df.isnull().sum()/ df.shape[0]).sort_values(ascending=False))

missing_val(bid_data_click)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 236898 entries, 2022-10-31 to 2023-01-03
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            236898 non-null  int64  
 1   Impression       236898 non-null  int64  
 2   Click            236898 non-null  int64  
 3   Cost             236898 non-null  float64
 4   Sum of AD rank   236898 non-null  float64
 5   Bid              236898 non-null  int64  
 6   Price(PC)        236898 non-null  int64  
 7   Price(Mobile)    236898 non-null  int64  
 8   Delivery Fee     236898 non-null  int64  
 9   Category Name 1  236898 non-null  object 
 10  Category Name 2  236898 non-null  object 
 11  Category Name 3  236898 non-null  object 
 12  Category Name 4  236898 non-null  object 
 13  CTR              236898 non-null  float64
 14  CPM              236898 non-null  float64
dtypes: float64(4), int64(7), object(4)
memory usage: 28.9+ MB
None


Feat

In [12]:
bid_data_click["Year"] = bid_data_click.index.year 
bid_data_click["Quarter"] = bid_data_click.index.quarter
bid_data_click["Month"] = bid_data_click.index.month
bid_data_click["Week"] = bid_data_click.index.week
bid_data_click["Weekday"] = bid_data_click.index.weekday
bid_data_click["Day"] = bid_data_click.index.day
bid_data_click["Dayofyear"] = bid_data_click.index.dayofyear
bid_data_click["Date"] = pd.DatetimeIndex(bid_data_click.index).date

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\2055812707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\2055812707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\2055812707.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [13]:
date_list = list(set(bid_data_click['Date'].to_list()))
print((date_list))

[datetime.date(2022, 11, 3), datetime.date(2022, 11, 9), datetime.date(2022, 12, 27), datetime.date(2022, 11, 18), datetime.date(2022, 12, 28), datetime.date(2022, 11, 10), datetime.date(2022, 11, 29), datetime.date(2022, 11, 22), datetime.date(2022, 11, 26), datetime.date(2022, 12, 16), datetime.date(2022, 11, 23), datetime.date(2022, 12, 29), datetime.date(2022, 11, 27), datetime.date(2022, 11, 19), datetime.date(2022, 12, 30), datetime.date(2022, 12, 4), datetime.date(2022, 11, 6), datetime.date(2022, 12, 13), datetime.date(2022, 11, 4), datetime.date(2022, 12, 19), datetime.date(2022, 11, 14), datetime.date(2022, 12, 18), datetime.date(2022, 11, 11), datetime.date(2022, 11, 17), datetime.date(2022, 11, 21), datetime.date(2022, 11, 16), datetime.date(2022, 11, 25), datetime.date(2022, 11, 30), datetime.date(2022, 12, 10), datetime.date(2022, 11, 24), datetime.date(2022, 12, 11), datetime.date(2022, 12, 26), datetime.date(2022, 12, 14), datetime.date(2022, 12, 25), datetime.date(2022

In [14]:
columns = ['ds', 'y']
bid_cost_mean = pd.DataFrame(columns=columns)
for date in date_list:
    bid_mean = bid_data_click.loc[bid_data_click['Date'] == date]
    bid_cost_mean = bid_cost_mean.append(pd.DataFrame([[date, bid_mean['Cost'].mean()]], columns=columns), ignore_index=True)
bid_cost_mean

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11996\29616653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

,ds,y
0,2022-11-03,548.929876
1,2022-11-09,1189.766867
2,2022-12-27,433.375517
3,2022-11-18,726.830244
4,2022-12-28,437.153539
5,2022-11-10,778.717579
6,2022-11-29,590.000000
7,2022-11-22,1211.584273
8,2022-11-26,1044.544413
9,2022-12-16,808.916667


In [15]:
class Prophet_Class:
    
    def __init__(self, df: pd.DataFrame, target: str, param_grid: dict): 
        
        
        self.df = df
        
        self.df['y'] = target
        
        self.df['ds'] = df['ds']
    
        self.param_grid = param_grid
        

    
    
    def metrics(self, metrix: str, horizon: str):

    # horizon 사용시 e.g.'365 days'
    # metrix = mse rmse mae mdape smape coverage 

        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
        evaluation_metrix = []  

        for params in all_params:

            model = Prophet(**params)
            model.fit(df)  

            df_cv = cross_validation(model, horizon=horizon, parallel="processes")

            df_p = performance_metrics(df_cv)

            evaluation_metrix.append(df_p[metrix].values[0])

        tuning_results = pd.DataFrame(all_params)
        tuning_results[metrix] = evaluation_metrix


        tuning_results = tuning_results.sort_values(by=[metrix])


        self.final_params = tuning_results.iloc[0, :-1].to_dict()




        def final(periods: int, freq: str):

            m = Prophet(**final_params).fit(df)
            future = m.make_future_dataframe(periods=periods, freq=freq)
            forecast = m.predict(future)


            return final_params, forecast.loc[df.shape[0]:, 'yhat'].values




In [16]:
bid_cost_mean

,ds,y
0,2022-11-03,548.929876
1,2022-11-09,1189.766867
2,2022-12-27,433.375517
3,2022-11-18,726.830244
4,2022-12-28,437.153539
5,2022-11-10,778.717579
6,2022-11-29,590.000000
7,2022-11-22,1211.584273
8,2022-11-26,1044.544413
9,2022-12-16,808.916667


In [17]:
param_grid = {
              'n_changepoints' : [25], 
              'changepoint_range' : [0.8],   
              'seasonality_prior_scale' : [20], 
              'changepoint_prior_scale' : [0.05], 
              'mcmc_samples' : [0], 
              'interval_width' : [0.8], 
              'uncertainty_samples' : [1000] 
    

}

In [18]:
engine = Prophet_Class(df=bid_cost_mean, target=bid_cost_mean['y'], param_grid=param_grid)

In [19]:
engine.metrics(metrix = 'mse', horizon = '7 days')

NameError: name 'df' is not defined